In [8]:
import multiprocessing
from pathlib import Path
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import cv2
import numpy as np
import imutils
import random
import pickle
import os
import shutil
import time

In [9]:
ROOT_DIR = Path.cwd().parent

SHAPES = ['Cercles', 'Diamants', 'Hexagones', 'Triangles']

THRESHOLDS = {
    'Cercle2': 0.5,
    'Cercle3': 0.5,
    'Cercle4': 0.5,
    'Cercle5': 0.4,
    'Diamant2': 0.5,
    'Diamant3': 0.4,
    'Diamant4': 0.4,
    'Diamant5': 0.4,
    'Hexagone2': 0.5,
    'Hexagone3': 0.5,
    'Hexagone4': 0.5,
    'Hexagone5': 0.3,
    'Triangle2': 0.5,
    'Triangle3': 0.5,
    'Triangle4': 0.5,
    'Triangle5': 0.5,
}


GROUPS = [2,3,4,5]

path_ensemble_A = Path(ROOT_DIR, 'EnsembleA_H2020')

In [10]:
def get_shapes_markers(shape_dir):
    """
    Prend le nom du pattern en string en parametre
    :return dict avec les paths des images et le marker associer
    exemple:
        {
            "Cercle2":  {
                "marker_path": PosixPath('path/to/markers/C2.jpg'),
                "images": [PosixPath('path/to/images/Cercle2/001_Cercle2.jpg'), ...],
            },
            "Cercle3": {...},
            ...
        }
    """
    paths = [path_ensemble_A/Path(shape_dir, f'{shape_dir[:-1]}{i}') for i in GROUPS]

    sub_shapes = dict()
    for p in paths:
        img = []
        for root, dirs, files in os.walk(str(p)):
            [img.append(f'{p}/{f}') for f in files]
        sub_shapes.update({
            p.name: {
                'marker_path': Path(ROOT_DIR, 'resources', 'markers', f'{p.name[:1]+p.name[-1:]}.jpg'),
                'images': img,
            }
        })

    return sub_shapes

In [11]:
# RUN THIS CELL TO CREATE ENSEMBLE_B DATABASE
#  TO REVISIT:
#  TWEEKING OF THRESHOLD FOR EACH SHAPE 

ensemble_A = {}

for shape in SHAPES:
    imgs = get_shapes_markers(shape)
    ensemble_A.update({shape: imgs})
    
path_ensemble_B = Path(ROOT_DIR, 'EnsembleB_H2020')

if os.path.exists(str(path_ensemble_B)):
    shutil.rmtree(str(path_ensemble_B), ignore_errors=True)
    time.sleep(1)
path_ensemble_B.mkdir(parents=True, exist_ok=True)


In [12]:
for shape, groups in ensemble_A.items():
    for group, paths in groups.items():
        group_path = path_ensemble_B/Path(shape, group)
        group_path.mkdir(parents=True, exist_ok=True)
        marker = cv2.imread(str(paths['marker_path']), cv2.IMREAD_GRAYSCALE)
        nb_match = 0
        for image_path in paths['images']:
            image = cv2.imread(str(image_path))
            new_path=str(image_path).replace("EnsembleA","EnsembleB")

            bitwise = cv2.bitwise_not(image)
            resized = imutils.resize(bitwise, width=300)
            ratio = image.shape[0] / float(resized.shape[0])

            #  https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_thresholding/py_thresholding.html
            # convert the resized image to grayscale, blur it slightly,
            # and threshold it
            gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
            # blurred = cv2.GaussianBlur(gray, (5, 5), 0)
            # thresh = cv2.threshold(blurred, 160, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C)[1]

# WILL NEED TO EXTRACT IMAGE BEFORE FINDING FEATURES
            img = gray
            w,h = img.shape[::-1]
            base_width = int(w/2)
            i=0
            has_matched = False
            templateRotation=0
#             im_crop=0
            while templateRotation < 350:

                rotated = imutils.rotate(marker, angle=templateRotation)
                templateRotation=templateRotation+10

                while i < base_width-40:
                    if has_matched:
                        break
                    bitwise = cv2.bitwise_not(marker)
                    resized = imutils.resize(bitwise, width=base_width-i)

                    w,h = resized.shape[::-1]
                    i=i+20
                    result = cv2.matchTemplate(img, resized, cv2.TM_CCOEFF_NORMED)
                    loc = np.where(result >= THRESHOLDS[group])

                    for pt in zip(*loc[::-1]):
                        has_matched = True
                        im_crop = img[pt[1]:pt[1]+h, pt[0]:pt[0]+w]
                        cv2.imwrite(new_path, im_crop)
                        nb_match = nb_match + 1
                        break
                        
        print(f'match {group} : {nb_match}/{len(paths["images"])}')
            